In [404]:
%pip install setuptools
%pip install ydata_profiling
%pip install scikit-learn
%pip install xlrd
%pip install openpyxl
%pip install ipywidgets
%pip install statsmodels
%pip install catboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [405]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from ydata_profiling import ProfileReport
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.tsa.stattools import adfuller
from catboost import CatBoostRegressor
import catboost
from catboost import *
import statsmodels.api as sm

In [406]:
excel_data=pd.read_excel("././dataset/Объёмы перевозок.xls", skiprows=[0], header=[0,1])
excel_data_volumes = excel_data

In [407]:
excel_data_volumes.sort_index(axis=1, inplace=True)
num_col=len(excel_data_volumes.columns)
left = excel_data_volumes.iloc[:, [num_col-5, num_col-4, num_col-3, num_col-2,num_col-1]]
left.columns = left.columns.get_level_values(1)
left = left.set_axis(["id", "subj_from", "subj_to", "cargo_code", "cargo_type"], axis = 1)

In [408]:
labelencodder = LabelEncoder()
subj = pd.concat([left.loc[:, ["subj_from"]].rename(columns = {"subj_from": "subj"}), left.loc[:, ["subj_to"]].rename(columns = {"subj_to": "subj"})]).drop_duplicates(ignore_index=True)
cargo_type = left.loc[:, ["cargo_type"]].drop_duplicates(ignore_index=True)
subj.to_excel("././dataset/subj.xlsx")
cargo_type.to_excel("././dataset/cargo_type.xlsx")

In [409]:
left.loc[:, ["subj_from"]] = left.loc[:, ["subj_from"]].apply(lambda x: subj[subj["subj"] == x["subj_from"]].index[0], axis=1)
left.loc[:, ["subj_to"]] = left.loc[:, ["subj_to"]].apply(lambda x: subj[subj["subj"] == x["subj_to"]].index[0], axis=1)
left.loc[:, ["cargo_type"]] = left.loc[:, ["cargo_type"]].apply(lambda x: cargo_type[cargo_type["cargo_type"] == x["cargo_type"]].index[0], axis=1)

In [410]:
left["subj_from"] = left["subj_from"].astype("int64")
left["subj_to"] = left["subj_to"].astype("int64")
left["cargo_type"] = left["cargo_type"].astype("int64")

In [411]:
for i in range(int((num_col-5)/2)):
    right = excel_data_volumes.iloc[:, [i*2,i*2+1]]
    date = pd.to_datetime(excel_data_volumes.iloc[:, [i*2]].columns.get_level_values(0).to_list()[0], format="%Y/%m")
    
    right.insert(2, ("", "date"), date)
    right.insert(3, ("", "month"), date.month)
    right.insert(4, ("", "quarter"), date.quarter)
    right.insert(5, ("", "year"), date.year)
    right.columns = right.columns.get_level_values(1)
    right = right.set_axis(["cargo_volume", "cargo_sales", "date", "month", "quarter", "year"], axis=1)
    right.insert(2, ("cargo_price"), right.apply(lambda x: (x["cargo_sales"] / x["cargo_volume"]) if x["cargo_volume"] > 0 else x["cargo_sales"], axis= 1))
    if i == 0:
        result = pd.concat([left, right], axis = 1, sort = False)
    else:
        result = pd.concat([result, pd.concat([left, right], axis = 1, sort = False)], axis = 0, ignore_index=True, sort=False)
excel_data_volumes = result

In [412]:
excel_data_volumes.insert(12, ("month_of_period"), 0)

In [413]:
min = excel_data_volumes["year"].min()
excel_data_volumes["month_of_period"] = excel_data_volumes["year"].apply(lambda x: x - min)*12 + excel_data_volumes["month"] 
excel_data_volumes = excel_data_volumes.drop("date", axis = 1)

In [414]:
excel_data_volumes.to_excel("././dataset/volumes.xlsx")

In [415]:
excel_data_volumes

,id,subj_from,subj_to,cargo_code,cargo_type,cargo_volume,cargo_sales,cargo_price,month,quarter,year,month_of_period
0,21220,0,12,39146,0,0,0,0.000000,1,1,2022,1
1,912,1,13,32411,1,0,0,0.000000,1,1,2022,1
2,912,1,14,32411,1,476,1440050,3025.315126,1,1,2022,1
3,912,1,6,32411,1,0,0,0.000000,1,1,2022,1
4,912,1,12,32411,1,0,0,0.000000,1,1,2022,1
...,...,...,...,...,...,...,...,...,...,...,...,...
429595,11274,2,34,24116,2,136,81935,602.463235,8,3,2024,32
429596,20123,6,12,32411,1,397,871122,2194.261965,8,3,2024,32
429597,20123,6,17,32411,1,135,461477,3418.348148,8,3,2024,32
429598,20123,6,40,32411,1,600,1342809,2238.015000,8,3,2024,32


In [416]:
excel_data=pd.read_excel("././dataset/MC_1.xls", header=[0])
excel_data_MC = excel_data
excel_data=pd.read_excel("././dataset/MC_2.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_4.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_5.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_6.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_7.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)
excel_data=pd.read_excel("././dataset/MC_8.xls", header=[0])
excel_data_MC = pd.concat([excel_data_MC, excel_data], axis=0, ignore_index=True, sort=False)

In [417]:
excel_data_MC = excel_data_MC.drop(excel_data_MC.columns[[3,4,6,7,8,13,15,17]], axis = 1).drop_duplicates()
excel_data_MC = excel_data_MC.set_axis(["id", "is_msp", "company_size", "company_okved2", "els", "is_otpr", "is_poluch", "index_pl", "index_fin", "kontrakt_type"], axis=1)

In [418]:
excel_data_MC["is_msp"] = excel_data_MC["is_msp"].apply(lambda x: 1 if x == "Да" else 0)
excel_data_MC["is_otpr"] = excel_data_MC["is_otpr"].apply(lambda x: 1 if x == "Да" else 0) 
excel_data_MC["is_poluch"] = excel_data_MC["is_poluch"].apply(lambda x: 1 if x == "Да" else 0) 

In [419]:
excel_data_MC["company_size"] = excel_data_MC["company_size"].apply(lambda x: "Неизвестно" if x is np.nan else x)
excel_data_MC["company_okved2"] = excel_data_MC["company_okved2"].apply(lambda x: "0" if x is np.nan else x)
excel_data_MC["els"] = excel_data_MC["els"].apply(lambda x: 0 if x is np.nan else 1)
excel_data_MC["index_pl"] = excel_data_MC["index_pl"].apply(lambda x: "Недостаточно данных для расчета риска просрочки платежа" if x is np.nan else x)
excel_data_MC["index_fin"] = excel_data_MC["index_fin"].apply(lambda x: "Недостаточно данных для расчета индекса финансового риска" if x is np.nan else x)
excel_data_MC["kontrakt_type"] = excel_data_MC["kontrakt_type"].apply(lambda x: "Неизвестно" if x is np.nan else x)

In [420]:
company_size = excel_data_MC.loc[:, ["company_size"]].drop_duplicates(ignore_index=True)
company_size.to_excel("././dataset/company_size.xlsx")
index_pl = excel_data_MC.loc[:, ["index_pl"]].drop_duplicates(ignore_index=True)
index_pl.to_excel("././dataset/index_pl.xlsx")
index_fin = excel_data_MC.loc[:, ["index_fin"]].drop_duplicates(ignore_index=True)
index_fin.to_excel("././dataset/index_fin.xlsx")
kontrakt_type = excel_data_MC.loc[:, ["kontrakt_type"]].drop_duplicates(ignore_index=True)
kontrakt_type.to_excel("././dataset/kontrakt_type.xlsx")

In [421]:
excel_data_MC.loc[:, ["company_size"]] = excel_data_MC.loc[:, ["company_size"]].apply(lambda x: company_size[company_size["company_size"] == x["company_size"]].index[0], axis=1)
excel_data_MC.loc[:, ["index_pl"]] = excel_data_MC.loc[:, ["index_pl"]].apply(lambda x: index_pl[index_pl["index_pl"] == x["index_pl"]].index[0], axis=1)
excel_data_MC.loc[:, ["index_fin"]] = excel_data_MC.loc[:, ["index_fin"]].apply(lambda x: index_fin[index_fin["index_fin"] == x["index_fin"]].index[0], axis=1)
excel_data_MC.loc[:, ["kontrakt_type"]] = excel_data_MC.loc[:, ["kontrakt_type"]].apply(lambda x: kontrakt_type[kontrakt_type["kontrakt_type"] == x["kontrakt_type"]].index[0], axis=1)

In [422]:
excel_data_MC = excel_data_MC.drop_duplicates()

In [423]:
company_okved2 = excel_data_MC.groupby("id").apply(lambda x: list(set(list(x["company_okved2"]))), include_groups=False)
company_okved2 = company_okved2.reset_index().set_axis(["id", "company_okved2"], axis=1)  

In [424]:
is_msp = excel_data_MC.groupby("id").apply(lambda x: max(x["is_msp"]), include_groups=False)
is_msp = is_msp.reset_index().set_axis(["id", "is_msp"], axis=1) 

In [425]:
company_size = excel_data_MC.groupby("id").apply(lambda x: max(x["company_size"]), include_groups=False)
company_size = company_size.reset_index().set_axis(["id", "company_size"], axis=1)     

In [426]:
els = excel_data_MC.groupby("id").apply(lambda x: max(x["els"]), include_groups=False)
els = els.reset_index().set_axis(["id", "els"], axis=1)  

In [427]:
is_otpr = excel_data_MC.groupby("id").apply(lambda x: max(x["is_otpr"]), include_groups=False)
is_otpr = is_otpr.reset_index().set_axis(["id", "is_otpr"], axis=1)  

In [428]:
is_poluch = excel_data_MC.groupby("id").apply(lambda x: max(x["is_poluch"]), include_groups=False)
is_poluch = is_poluch.reset_index().set_axis(["id", "is_poluch"], axis=1) 

In [429]:
index_pl = excel_data_MC.groupby("id").apply(lambda x: max(x["index_pl"]), include_groups=False)
index_pl = index_pl.reset_index().set_axis(["id", "index_pl"], axis=1) 

In [430]:
index_fin = excel_data_MC.groupby("id").apply(lambda x: max(x["index_fin"]), include_groups=False)
index_fin = index_fin.reset_index().set_axis(["id", "index_fin"], axis=1) 

In [431]:
kontrakt_type = excel_data_MC.groupby("id").apply(lambda x: list(set(list(x["kontrakt_type"]))), include_groups=False)
kontrakt_type = kontrakt_type.reset_index().set_axis(["id", "kontrakt_type"], axis=1) 

In [432]:
id = excel_data_MC["id"].drop_duplicates()
id = id.reset_index().drop(columns=["index"])

In [433]:
excel_data_MC.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9512 entries, 0 to 183206
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              9512 non-null   int64 
 1   is_msp          9512 non-null   int64 
 2   company_size    9512 non-null   object
 3   company_okved2  9512 non-null   object
 4   els             9512 non-null   int64 
 5   is_otpr         9512 non-null   int64 
 6   is_poluch       9512 non-null   int64 
 7   index_pl        9512 non-null   object
 8   index_fin       9512 non-null   object
 9   kontrakt_type   9512 non-null   object
dtypes: int64(5), object(5)
memory usage: 817.4+ KB


In [434]:
excel_data = pd.merge(id, is_msp, left_on="id", right_on="id")


In [435]:
excel_data = pd.merge(excel_data, company_size, left_on="id", right_on="id")

In [436]:
excel_data = pd.merge(excel_data, company_okved2, left_on="id", right_on="id")

In [437]:
excel_data = pd.merge(excel_data, els, left_on="id", right_on="id")

In [438]:
excel_data = pd.merge(excel_data, is_otpr, left_on="id", right_on="id")

In [439]:
excel_data = pd.merge(excel_data, is_poluch, left_on="id", right_on="id")

In [440]:
excel_data = pd.merge(excel_data, index_pl, left_on="id", right_on="id")

In [441]:
excel_data = pd.merge(excel_data, index_fin, left_on="id", right_on="id")

In [442]:
excel_data = pd.merge(excel_data, kontrakt_type, left_on="id", right_on="id")

In [443]:
excel_data_MC = excel_data

In [444]:
excel_data_MC.to_excel("././dataset/MC.xlsx")

In [466]:
excel_data_MC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9267 entries, 0 to 9266
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              9267 non-null   int64 
 1   is_msp          9267 non-null   int64 
 2   company_size    9267 non-null   int64 
 3   company_okved2  9267 non-null   object
 4   els             9267 non-null   int64 
 5   is_otpr         9267 non-null   int64 
 6   is_poluch       9267 non-null   int64 
 7   index_pl        9267 non-null   int64 
 8   index_fin       9267 non-null   int64 
 9   kontrakt_type   9267 non-null   object
dtypes: int64(8), object(2)
memory usage: 724.1+ KB


In [446]:
data_pd = excel_data_volumes.drop("subj_from", axis = 1)
data_pd = data_pd.drop("subj_to", axis = 1)
data_pd = data_pd.drop("cargo_code", axis = 1)
data_pd = data_pd.drop("cargo_type", axis = 1)


In [447]:
data_pd = data_pd.groupby(["id", "month_of_period", "year", "quarter", "month" ], as_index=False).sum()

In [448]:
data_pd["innactive"] = 0
data_pd["active"] = 0
data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd["active"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] > 0) & (x["cargo_sales"] > 0)) else 0, axis=1)

In [449]:
data_pd.head(10)

,id,month_of_period,year,quarter,month,cargo_volume,cargo_sales,cargo_price,innactive,active
0,9,1,2022,1,1,0,0,0.00000,1,0
1,9,2,2022,1,2,0,0,0.00000,1,0
2,9,3,2022,1,3,0,0,0.00000,1,0
3,9,4,2022,2,4,0,0,0.00000,1,0
4,9,5,2022,2,5,0,0,0.00000,1,0
5,9,6,2022,2,6,0,0,0.00000,1,0
6,9,7,2022,3,7,0,0,0.00000,1,0
7,9,8,2022,3,8,0,0,0.00000,1,0
8,9,9,2022,3,9,61,155364,2546.95082,0,1
9,9,10,2022,4,10,0,0,0.00000,1,0


In [450]:
data_pd["outflow"] = 0

def calc_innactive(row):
    prev_row_value = 0
    row_value = data_pd.iloc[row.name]["innactive"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["innactive"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("innactive")] = prev_row_value + 1
                row_value = data_pd.iloc[row.name]["innactive"]
                if row_value > 11:
                    prev_row_value = 1
                else:
                    prev_row_value = row_value/12
            else: 
                prev_row_value = 0
    return prev_row_value

def calc_active(row):
    row_value = data_pd.iloc[row.name]["active"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["active"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("active")] = prev_row_value + 1
    return 0

data_pd["outflow"] = data_pd.apply(calc_innactive, axis=1)
# data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd.apply(calc_active, axis=1)


0        0
1        0
2        0
3        0
4        0
        ..
30587    0
30588    0
30589    0
30590    0
30591    0
Length: 30592, dtype: int64

In [451]:
data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd["active"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] > 0) & (x["cargo_sales"] > 0)) else 0, axis=1)

In [452]:
excel_data = data_pd[data_pd["month_of_period"] == max(data_pd["month_of_period"])].loc[:, ["id", "outflow"]].reset_index()

In [453]:
excel_data = excel_data.drop(columns = ["index"])

In [454]:
excel_data["outflow"] = excel_data["outflow"].apply(lambda x: 1 if x ==1 else 0)

In [455]:
data_pd = data_pd.drop(columns = ["month_of_period", "year", "quarter", "month", "outflow"]).groupby(["id"]).sum()

In [456]:
data_pd["activity"] = data_pd.apply(lambda x: x["active"]/(x["active"]+x["innactive"]), axis = 1)

In [457]:
data_pd = data_pd.drop(columns=["innactive", "active"])

In [458]:
data_pd

,cargo_volume,cargo_sales,cargo_price,activity
id,,,,
9,216,663977,1.241603e+04,0.12500
25,186332,83893559,1.122309e+05,1.00000
33,157844,463772680,4.176032e+06,1.00000
86,1664,2817957,5.001043e+03,0.09375
115,162887,391321720,1.184122e+06,1.00000
...,...,...,...,...
21725,12,30200,5.033333e+03,0.06250
21726,244,737941,6.073666e+03,0.06250
21727,3183,8943212,5.624780e+03,0.03125


In [ ]:
data_pd = pd.merge(data_pd, excel_data, left_on="id", right_on="id")

In [468]:
data_pd

,id,cargo_volume,cargo_sales,cargo_price,activity,outflow
0,9,216,663977,1.241603e+04,0.12500,0
1,25,186332,83893559,1.122309e+05,1.00000,0
2,33,157844,463772680,4.176032e+06,1.00000,0
3,86,1664,2817957,5.001043e+03,0.09375,1
4,115,162887,391321720,1.184122e+06,1.00000,0
...,...,...,...,...,...,...
951,21725,12,30200,5.033333e+03,0.06250,0
952,21726,244,737941,6.073666e+03,0.06250,0
953,21727,3183,8943212,5.624780e+03,0.03125,0
954,21728,48,247392,2.061600e+04,0.12500,0


In [461]:
excel_data_MC

,id,is_msp,company_size,company_okved2,els,is_otpr,is_poluch,index_pl,index_fin,kontrakt_type
0,864,0,0,[0],0,0,0,0,0,[0]
1,865,0,0,[0],1,0,0,0,0,[0]
2,866,0,0,[0],1,0,1,0,0,[0]
3,867,0,0,[0],0,0,0,0,0,[0]
4,868,0,0,[0],0,0,0,0,0,[0]
...,...,...,...,...,...,...,...,...,...,...
9262,11186,0,0,[0],0,0,0,0,0,[0]
9263,11187,0,0,[0],0,0,0,0,0,[0]
9264,11188,0,0,[0],0,0,0,0,0,[0]
9265,11189,0,0,[0],0,0,0,0,0,[0]


In [471]:
excel_data = pd.merge(excel_data_MC, data_pd, left_on="id", right_on="id", how="outer")

In [477]:
excel_data.to_excel("././dataset/merge_data.xlsx")

In [480]:
excel_data

,id,is_msp,company_size,company_okved2,els,is_otpr,is_poluch,index_pl,index_fin,kontrakt_type,cargo_volume,cargo_sales,cargo_price,activity,outflow
0,0,0.0,4.0,[0],1.0,0.0,0.0,0.0,0.0,[0],NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,[0],1.0,0.0,0.0,0.0,0.0,[0],NaN,NaN,NaN,NaN,NaN
2,2,0.0,0.0,[0],1.0,0.0,0.0,0.0,0.0,[0],NaN,NaN,NaN,NaN,NaN
3,3,0.0,0.0,[0],0.0,0.0,0.0,0.0,0.0,[0],NaN,NaN,NaN,NaN,NaN
4,4,0.0,0.0,[0],0.0,0.0,0.0,0.0,0.0,[0],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,21725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,30200.0,5033.333333,0.06250,0.0
9995,21726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244.0,737941.0,6073.666304,0.06250,0.0
9996,21727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3183.0,8943212.0,5624.780072,0.03125,0.0
9997,21728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,247392.0,20616.000000,0.12500,0.0


In [ ]:
keep = data_pd['id'].drop_duplicates().sample(frac=0.7)
m = data_pd['id'].isin(keep)
train = data_pd[m]
test = data_pd[~m]

train.to_excel("train.xlsx")
test.to_excel("test.xlsx")

In [ ]:
data_pd.head(10)

In [ ]:
# Задаём размер визуализации
fig, ax = plt.subplots(figsize=(20, 5))
data = data_pd[data_pd["id"] == 424]
# Построение линейчатого графика временного ряда
ax.plot(data['month_of_period'], data['cargo_price'], label='Цена')

# Добавление линии регрессии с помощью Seaborn
sns.regplot(x='month_of_period', y='cargo_price', data=data, ci=None, scatter_kws=dict(color='0.25'), ax=ax)

ax.set_title('График цены перевозки')
ax.set_xlabel('Месяц')
ax.set_ylabel('Цена')
ax.legend()
plt.show() 

In [ ]:
data.loc[:, ["lag_1"]] = data['cargo_price'].shift(1)
data.loc[:, ["lag_2"]] = data['cargo_price'].shift(2)
data.loc[:, ["lag_3"]] = data['cargo_price'].shift(3)
data.loc[:, ["lag_4"]] = data['cargo_price'].shift(4)
data.loc[:, ["lag_5"]] = data['cargo_price'].shift(5)
data.loc[:, ["lag_6"]] = data['cargo_price'].shift(6)
data.loc[:, ["out"]]  = data.apply(lambda x: 1 if (x["outflow"] == 1) else 0, axis=1).astype("int64")

In [ ]:
data.dropna(inplace=True)
data

In [ ]:
data.info()

In [ ]:

fig, ax = plt.subplots() 
ax = sns.regplot(x='lag_1', y='cargo_price', data=data, ci=None, scatter_kws=dict(color='0.25'))

# Устанавливаем заголовок и подписи осей
ax.set_title('Диаграмма рассеяния с lag признаком (запаздывание 1)')
ax.set_xlabel('Цена с запаздыванием (lag-1)')
ax.set_ylabel('Цена')

plt.show()

In [ ]:
data.set_index('month_of_period', inplace=True)

In [ ]:
# Декомпозиция временного ряда
decomposition = sm.tsa.seasonal_decompose(data['cargo_price'], model='additive', period=13)

# Визуализация компонентов временного ряда
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(15, 8))

# Исходный ряд
decomposition.observed.plot(ax=ax1)
ax1.set_ylabel('Цена')
ax1.set_title('Исходный ряд')

# Тренд
decomposition.trend.plot(ax=ax2)
ax2.set_ylabel('Цена')
ax2.set_title('Тренд')

# Сезонность
decomposition.seasonal.plot(ax=ax3)
ax3.set_ylabel('Цена')
ax3.set_title('Сезонность')

# Остатки
decomposition.resid.plot(ax=ax4)
ax4.set_ylabel('Цена')
ax4.set_title('Нерегулярность')

plt.tight_layout()
plt.show()

In [ ]:
data['cargo_price']

# ADF-тест
result_adf = adfuller(data['cargo_price'])
print('ADF Test:')
print(f'ADF Statistic: {result_adf[0]}')
print(f'p-value: {result_adf[1]}')
print(f'Critical Values:')
for key, value in result_adf[4].items():
    print(f'   {key}: {value}')

In [ ]:
from statsmodels.tsa.stattools import kpss

# KPSS-тест
result_kpss = kpss(data['cargo_price'], regression='c')
print('KPSS Test:')
print(f'KPSS Statistic: {result_kpss[0]}')
print(f'p-value: {result_kpss[1]}')
print(f'Critical Values:')
for key, value in result_kpss[3].items():
    print(f'   {key}: {value}')

In [ ]:
# Установим окно равное 12
rolling_mean = data['cargo_price'].rolling(window=12).mean()

# Детерминируем тренд
detrended = data['cargo_price'] - rolling_mean

plt.figure(figsize=(14, 7))
plt.plot(data['cargo_price'], label='Исходный ряд')
plt.plot(rolling_mean, label='Скользящее среднее', color='red')
plt.plot(detrended, label='Detrended', color='green')
plt.title('Детерминированный тренд')
plt.xlabel('Время')
plt.ylabel('Цена')
plt.grid(False) 
plt.legend()
plt.show()

In [ ]:
differenced = data['cargo_price'].diff().dropna()

plt.figure(figsize=(14, 7))
plt.plot(data['cargo_price'], label='Исходный ряд')
plt.plot(differenced, label=' Дифференцированный ряд', color='green')
plt.title('Дифференцирование')
plt.grid(False) 
plt.legend()
plt.show()